In [ ]:
# Step 1: Import necessary libraries
import os
import cv2
import numpy as np
import pickle
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Step 2: Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Step 3: Function to extract faces using MTCNN
detector = MTCNN()

def extract_face(image, required_size=(160, 160)):
    results = detector.detect_faces(image)
    if len(results) == 0:
        return None
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    return face

# Step 4: Load and augment images
def load_images_from_folder(folder, augment=False):
    images = []
    labels = []
    for person_name in os.listdir(folder):
        person_folder = os.path.join(folder, person_name)
        if not os.path.isdir(person_folder):
            continue
        for filename in os.listdir(person_folder):
            img_path = os.path.join(person_folder, filename)
            img = cv2.imread(img_path)
            face = extract_face(img)
            if face is not None:
                images.append(face)
                labels.append(person_name)
                if augment:
                    face = face.reshape((1, ) + face.shape)
                    for i, batch in enumerate(datagen.flow(face, batch_size=1)):
                        augmented_img = batch[0].astype('uint8')
                        images.append(augmented_img)
                        labels.append(person_name)
                        save_augmented_image(augmented_img, person_name, filename, i)
                        if i >= 4:  # Augment 5 images per original image
                            break
    return np.array(images), np.array(labels)

def save_augmented_image(image, person_name, original_filename, idx):
    output_folder = os.path.join("augmented_data", person_name)
    os.makedirs(output_folder, exist_ok=True)
    base_name = os.path.splitext(original_filename)[0]
    output_path = os.path.join(output_folder, f"{base_name}_aug_{idx}.png")
    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Step 5: Load dataset with augmentation
dataset_path = "data"
images, labels = load_images_from_folder(dataset_path, augment=True)

# Step 6: Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

# Step 7: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Step 8: Define the model
model = Sequential([
    Flatten(input_shape=(160, 160, 3)),
    Dense(100, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 9: Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Step 10: Save the model using pickle
model_path = "face_recognition_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

# Step 11: Load the model using pickle
with open(model_path, 'rb') as f:
    model = pickle.load(f)

# Step 12: Real-time face recognition
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    faces = detector.detect_faces(frame)
    for face in faces:
        x, y, width, height = face['box']
        x, y = abs(x), abs(y)
        x2, y2 = x + width, y + height
        face_crop = frame[y:y2, x:x2]
        face_crop = cv2.resize(face_crop, (160, 160))
        face_crop = face_crop.astype('float32') / 255.0
        face_crop = np.expand_dims(face_crop, axis=0)
        
        # Predict the class
        prediction = model.predict(face_crop)
        class_index = np.argmax(prediction)
        class_name = label_encoder.classes_[class_index]
        
        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 13: Print necessary details for reference
print("Dataset Path:", dataset_path)
print("Number of Original Images:", len(os.listdir(dataset_path)))
print("Number of Augmented Images:", len(images) - len(os.listdir(dataset_path)))
print("Model Path:", model_path)
print("Classes:", label_encoder.classes_)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/s